# 1. 로봇 이벤트 데이터 생성기
<img src="generator.png" alt="generator" width="500"/>

이 스크립트는 로봇 이벤트 데이터를 JSON Lines 형식으로 생성하여 파일로 저장합니다.

## 주요 특징
- **로봇 ID**: `robot_001` ~ `robot_005`
- **명령 종류**: `move_to_point`, `pick_item`, `place_item`, `charge`, `scan_area`
- **샘플 개수**: 10,000
- **출력 파일**: `sample_json_output/robot_data_10000.json`

## 이벤트 구조
- **event_id**: UUID 형식의 고유 이벤트 ID
- **robot_id**: 로봇 식별자
- **timestamp**: ISO8601 UTC 시간
- **location**: x, y, z 좌표 (이동 반영)
- **event**:
  - `event_type`: `"command_execution"`
  - `command`: 명령 이름, 목표 좌표, 실행 시간(ms), 성공 여부
- **sensors**:
  - 배터리: 잔량(%), 온도(℃)
  - LiDAR: 장애물 감지, 0/90/180/270° 방향 거리 측정

## 동작 방식
1. 초기 위치 (0,0,0)에서 시작
2. 랜덤 명령 실행 및 위치 갱신
3. 배터리/LiDAR 센서 값 무작위 생성
4. 각 이벤트를 JSON으로 변환하여 한 줄씩 저장
5. 이벤트 간 시간 간격: **5~10초**

## 결과
- 지정한 개수(`10,000`)만큼 이벤트 데이터가 파일에 생성 및 저장됨

In [0]:
import json
import random
import uuid
from datetime import datetime, timedelta
import os

# Basic settings
robot_ids = [f"robot_{i:03}" for i in range(1, 6)]
commands = ["move_to_point", "pick_item", "place_item", "charge", "scan_area"]
start_time = datetime(2025, 7, 25, 9, 0, 0)
sample_count = 10_000
output_dir = "sample_json_output"
output_file = "robot_data_10000.json"

# Initial location settings
locations = {r: {"x": 0.0, "y": 0.0, "z": 0.0} for r in robot_ids}
random.seed(42)

os.makedirs(output_dir, exist_ok=True)

# Data generation function
def generate_event(timestamp, robot_id, prev_location):
    command = random.choice(commands)
    duration = random.randint(500, 8000)  # ms
    success = random.random() > 0.1

    # Move location
    delta_x = random.uniform(-5, 5)
    delta_y = random.uniform(-5, 5)
    new_location = {
        "x": round(prev_location["x"] + delta_x, 2),
        "y": round(prev_location["y"] + delta_y, 2),
        "z": 0.0
    }

    return {
        "event_id": str(uuid.uuid4()),
        "robot_id": robot_id,
        "timestamp": timestamp.isoformat() + "Z",
        "location": new_location,
        "event": {
            "event_type": "command_execution",
            "command": {
                "name": command,
                "parameters": {
                    "target_x": round(new_location["x"] + random.uniform(-1, 1), 2),
                    "target_y": round(new_location["y"] + random.uniform(-1, 1), 2)
                },
                "duration_ms": duration,
                "success": success
            }
        },
        "sensors": {
            "battery": {
                "level_percent": round(max(0, 100 - 0.05 * duration + random.uniform(-2, 2)), 1),
                "temperature_c": round(random.uniform(25, 45), 1)
            },
            "lidar": {
                "obstacle_detected": random.random() < 0.2,
                "scan_points": [
                    {"angle": a, "distance": round(random.uniform(0.5, 5.0), 2)}
                    for a in range(0, 360, 90)
                ]
            }
        }
    }, new_location

# Generate and save
with open(os.path.join(output_dir, output_file), "w") as f:
    current_time = start_time
    for i in range(sample_count):
        robot_id = random.choice(robot_ids)
        prev_loc = locations[robot_id]
        record, new_loc = generate_event(current_time, robot_id, prev_loc)
        f.write(json.dumps(record) + "\n")
        locations[robot_id] = new_loc
        # Increase time by 5~10 seconds
        current_time += timedelta(seconds=random.randint(5, 10))

print(f"✅ Generation complete: {sample_count} records → '{output_dir}/{output_file}'")

# 2. Autoloader 테스트용 단일 JSON 생성기

이 스크립트는 **Databricks Autoloader** 테스트를 위해 개별 JSON 파일을 생성합니다.  
기존 데이터 디렉토리에 새 파일을 추가하여 Autoloader의 신규 파일 감지 동작을 검증할 수 있습니다.

## 주요 특징
- 단일 이벤트 JSON 파일 생성 (예: `json2.json`, `json3.json`)
- 명령 및 센서 데이터 무작위 포함
- 출력 경로: `sample_json_output/`

## 이벤트 구조
- **event_id**: UUID 형식의 고유 이벤트 ID
- **robot_id**: 로봇 식별자
- **timestamp**: ISO8601 UTC 시간
- **location**: x, y, z 좌표
- **event**:
  - 명령 이름, 목표 좌표, 실행 시간(ms), 성공 여부
- **sensors**:
  - 배터리 잔량(%), 온도(℃)
  - LiDAR 장애물 감지 및 4방향 거리 측정

In [0]:
import json
import random
import uuid
from datetime import datetime
import os

def generate_single_json(filename, timestamp, robot_id):
    commands = ["move_to_point", "pick_item", "place_item", "charge", "scan_area"]
    command = random.choice(commands)
    duration = random.randint(800, 6000)
    success = random.random() > 0.1

    location = {
        "x": round(random.uniform(0, 100), 2),
        "y": round(random.uniform(0, 100), 2),
        "z": 0.0
    }

    record = {
        "event_id": str(uuid.uuid4()),
        "robot_id": robot_id,
        "timestamp": timestamp.isoformat() + "Z",
        "location": location,
        "event": {
            "event_type": "command_execution",
            "command": {
                "name": command,
                "parameters": {
                    "target_x": round(location["x"] + random.uniform(-1, 1), 2),
                    "target_y": round(location["y"] + random.uniform(-1, 1), 2)
                },
                "duration_ms": duration,
                "success": success
            }
        },
        "sensors": {
            "battery": {
                "level_percent": round(random.uniform(50, 95), 1),
                "temperature_c": round(random.uniform(25, 40), 1)
            },
            "lidar": {
                "obstacle_detected": random.random() < 0.2,
                "scan_points": [
                    {"angle": a, "distance": round(random.uniform(0.5, 5.0), 2)}
                    for a in range(0, 360, 90)
                ]
            }
        }
    }

    output_dir = "sample_json_output"
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, filename), "w") as f:
        f.write(json.dumps(record) + "\n")

    print(f"✅ 생성 완료: {filename}")

# 실제 생성 실행
generate_single_json("json2.json", datetime(2025, 7, 26, 10, 0, 0), "robot_007")
generate_single_json("json3.json", datetime(2025, 7, 26, 11, 30, 0), "robot_008")
